In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import os
import gzip
import shutil
import struct
import urllib.request

In [3]:
import time

In [30]:
learning_rate = 0.01
batch_size = 128
n_epochs = 100
n_train = 60000
n_test = 10000

In [5]:
    #Download and unzip the dataset mnist if it's not already downloaded 
    #Download from http://yann.lecun.com/exdb/mnist
    mnist_folder = "C:\\Users\\USX28939\\PYTHON_CODE_BASE\\GitHub_Doc\\stanford-tensorflow-tutorials\\examples\\data\\MNST_DATA"

In [32]:
def download_one_file(download_url, 
                    local_dest, 
                    expected_byte=None, 
                    unzip_and_remove=False):
    """ 
    Download the file from download_url into local_dest
    if the file doesn't already exists.
    If expected_byte is provided, check if 
    the downloaded file has the same number of bytes.
    If unzip_and_remove is True, unzip the file and remove the zip file
    """
    if os.path.exists(local_dest) or os.path.exists(local_dest[:-3]):
        print('%s already exists' %local_dest)
    else:
        print('Downloading %s' %download_url)
        local_file, headers = urllib.request.urlretrieve(download_url, local_dest)
        file_stat = os.stat(local_dest)
        if expected_byte:
            if file_stat.st_size == expected_byte:
                print('Successfully downloaded %s' %local_dest)
                if unzip_and_remove:
                    with gzip.open(local_dest, 'rb') as f_in, open(local_dest[:-3],'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    os.remove(local_dest)
            else:
                print('The downloaded file has unexpected number of bytes')

In [35]:
import urllib
print (urllib.request.urlopen("http://yann.lecun.com/exdb/mnist\"))


SyntaxError: EOL while scanning string literal (<ipython-input-35-d986ab718bac>, line 2)

In [36]:
def download_mnist(path):
    """ 
    Download and unzip the dataset mnist if it's not already downloaded 
    Download from http://yann.lecun.com/exdb/mnist
    """
    os.mkdir(path)
    url = 'http://yann.lecun.com/exdb/mnist/'
    filenames = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
    expected_bytes = [9912422, 28881, 1648877, 4542]

    for filename, byte in zip(filenames, expected_bytes):
        download_url = os.path.join(url, filename)
        local_dest = os.path.join(path, filename)
        print(download_url,"    -> ", local_dest)
        download_one_file(download_url, local_dest, byte, True)

In [37]:
download_mnist(mnist_folder)

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz     ->  C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-images-idx3-ubyte.gz
Successfully downloaded C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz     ->  C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-labels-idx1-ubyte.gz
Successfully downloaded C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz     ->  C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\t10k-images-idx3-ubyte.gz
Successfully downloaded C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\M

In [6]:
def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k':
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte')
    #print(label_file )
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8) #int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1
    
    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols) #uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels

In [7]:
def read_mnist(path, flatten=True, num_train=55000):
    """
    Read in the mnist dataset, given that the data is stored in path
    Return two tuples of numpy arrays
    ((train_imgs, train_labels), (test_imgs, test_labels))
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0])
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return (train_img, train_labels), (val_img, val_labels), test


In [8]:
tf.reset_default_graph() 

In [9]:
train, val, test = read_mnist(mnist_folder, flatten=True)

In [10]:
print(len(train))
print(len(val))
print(len(test))

2
2
2


In [11]:
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

In [40]:
print(train[1])

[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [12]:
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

In [24]:
print(test_data)

<BatchDataset shapes: ((?, 784), (?, 10)), types: (tf.float32, tf.float64)>


In [14]:
print(train_data.output_types)
print(train_data.output_shapes)

(tf.float32, tf.float64)
(TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


In [15]:
iterator = tf.data.Iterator.from_structure(train_data.output_types,train_data.output_shapes)

In [16]:
img, label = iterator.get_next()

In [35]:
print(label)

Tensor("IteratorGetNext:1", shape=(?, 10), dtype=float64)


In [18]:
train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

In [44]:
w = tf.get_variable(name = "Weights", dtype = tf.float32, shape = (784,10), initializer = tf.random_normal_initializer(mean=0.0,
    stddev=1.0,seed=None,dtype=tf.float32))
b = tf.get_variable(name = "Bias", dtype = tf.float32, shape = (1,1) , initializer = tf.zeros_initializer())

ValueError: Variable Weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1470, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2956, in create_op
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [20]:
logits = tf.matmul(img,w) + b

In [21]:
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logits)
loss = tf.reduce_mean(entropy)

In [48]:
optimizer = tf.train.MomentumOptimizer(learning_rate=0.001, momentum = .1).minimize(loss)

In [49]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('C:/Users/USX28939/PYTHON_CODE_BASE/graphs', tf.get_default_graph())
# to access the tensorboard graph use tensorboard --logdir training:C:\path of the log dir
# there is no equal sign after the logdir and add any text before the drive as its a bug
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 12.348731806111891
Average loss epoch 1: 10.59124533409296
Average loss epoch 2: 9.430858217283737
Average loss epoch 3: 8.612977085557095
Average loss epoch 4: 7.985617655377055
Average loss epoch 5: 7.463224460912305
Average loss epoch 6: 7.012512007424998
Average loss epoch 7: 6.608266159545543
Average loss epoch 8: 6.242408138097718
Average loss epoch 9: 5.910529146638027
Average loss epoch 10: 5.6061500127925425
Average loss epoch 11: 5.326777762590453
Average loss epoch 12: 5.0718947865242185
Average loss epoch 13: 4.840120219629864
Average loss epoch 14: 4.6267435295637265
Average loss epoch 15: 4.431375542352366
Average loss epoch 16: 4.252670641278112
Average loss epoch 17: 4.088293953829034
Average loss epoch 18: 3.9375345884367476
Average loss epoch 19: 3.798849011021991
Average loss epoch 20: 3.6714196848314864
Average loss epoch 21: 3.552593929823055
Average loss epoch 22: 3.4432861699614414
Average loss epoch 23: 3.342257108799247
Average loss epoch 

In [26]:
import os as os

In [28]:
os.getcwd()

'C:\\Users\\USX28939\\PYTHON_CODE_BASE\\GitHub_Doc\\TensorFlow_Framework'